In [9]:
import shotgun_api3 as sa
import os
import imp

import datetime

from pprint import pprint

'''
Dummy Test
    
'''


sg = sa.Shotgun(
    'https://west.shotgunstudio.com',
    api_key='6h&ziahfuGbqdubxrxeyopinl',
    script_name = 'eventTrigger',
)


def create_seq( project, lev1, script_name = 'utils' ):
    key = [ ['name','is',project ] ]
    project = sg.find_one( 'Project',filters=key ) 
    lev1_key = [
                ['project','is',project],
                ['code','is',lev1],
            #    ['sg_prod_type','is',prod_type]
               ]
    lev1_ent = sg.find_one( 'Sequence', lev1_key )
    if lev1_ent:
        return lev1_ent
        #return 'err.exists'
    key = {
            'project'       : project,
            'code'          : lev1,
            #'sg_prod_type'  : prod_type,
           }
    result = sg.create( 'Sequence', key )
    return result


def create_shot( project, lev1, lev2, script_name = 'utils' ):
    key = [ ['name','is',project ] ]
    project = sg.find_one( 'Project',filters=key ) 
    
    lev1_key = [ ['project','is',project],['code','is',lev1] ]
    lev1_ent = sg.find_one( 'Sequence', lev1_key )
    if not lev1_ent:
        return 'err.seq'
    filters = [
                ['code','is',lev2],
                ['project','is',project ],
                ['sg_sequence','is',lev1_ent]
              ]
    shot = sg.find_one( 'Shot' ,filters )
    if shot:
        return 'err.exists'
    key = { 
            'project':project, 
            'code':lev2,
            'sg_sequence':lev1_ent, 
            #'sg_prod_type':prod_type,
           }
    shot = sg.create( 'Shot',key )
    return shot


def create_task( show, prod_type, lev_type, lev1, lev2, lev3,lev3_type,
                    script_name = 'eventTrigger' ):
    key = [ ['name','is',show ] ]
    project = sg.find_one( 'Project',filters=key ) 

    lev1_key = [ ['project','is',project],['code','is',lev1] ]

    if lev_type == 'shot':
        lev1_ent = sg.find_one( 'Sequence', lev1_key )
        if not lev1_ent:
            return 'err.lev1'
        lev2_key = [ 
                     ['sg_sequence','is',lev1_ent],
                    ['code','is',lev2],
                     #['sg_prod_type','is',prod_type] 
                    ]
        lev2_ent = sg.find_one( 'Shot' , lev2_key )
    else:
        lev2_key = [ 
                ['code', 'is', lev2], 
                ['sg_asset_type','is', ASSET_TYPE[lev1] ],
                ['project','is',project],
                #['sg_prod_type','is',prod_type] 
              ]
        lev2_ent = sg.find_one( 'Asset' ,lev2_key )

    if not lev2_ent:
        return 'err.lev2'

    task_key = [
                 ['project','is',project], 
                 [ 'entity','is',lev2_ent],
                 ['content','is',lev3 ]
               ]

    print '='*30
    print prod_type
    if prod_type == 'previz':
        task_key.append( [ 'step.Step.code','is','Previz' ] )
    else:
        task_key.append( [ 'step.Step.code','is_not','Previz' ] )

    lev3_ent = sg.find_one( 'Task', task_key )
    if not lev3_ent:
        if lev_type == 'shot':
            ent_type = 'Shot'
        else:
            ent_type = 'Asset'
        
        if prod_type == 'previz':
            step = sg.find_one( 'Step',[ ['code','is','Previz' ],
                                 ['entity_type','is',ent_type],
                               ] )
        else:
            step = sg.find_one( 'Step',[ ['code','is', lev3_type ],
                                 ['entity_type','is',ent_type],
                               ] )
        if not step:
            return 'err.step'

        key = { 'project' : project, 'entity':lev2_ent , 'step' : step ,
               'content':lev3 }

        task = sg.create( 'Task', key )
       
        result = sg.find_one( 'Task' , [['id','is',task['id']]],['entity','step','content',] )
        result['show'] = show
        result['lev_type'] = lev_type
        result['lev1'] = lev1
        result['lev2'] = lev2
        result['lev3'] = lev3
        result['createdBy'] = os.environ['USERNAME']
        result['datetime'] = datetime.datetime.now().strftime( '%Y-%m %H:%M:%S' )

        return result
    else:
        return 'err.exists'


print create_task( 'RND', 'post',  'shot', 'AAA','AAA_0030', 'anim', 'ani')
    
# print create_shot( 'RND', 'AAA','AAA_0030')

# print( create_seq( 'RND', 'AAA' , script_name= 'eventTrigger' ) )

post
err.exists


In [11]:
def create_version( show,  lev_type, lev2, lev3, basename, file_path,
                    login, path_to_frames = '', description='', status='wip',
                  script_name = 'evetTrigger' ):
    if lev_type == 'shot':
        lev = 'Shot'
    else:
        lev = 'Asset'

    ver_find = [
                    ['code','is', basename],
                    ['project.Project.name','is', show],
                    ['sg_task.Task.content', 'is', lev3],
                    ['sg_task.Task.entity.%s.code'% lev,'is', lev2],
#                     ['sg_task.Task.entity.%s.sg_prod_type'% lev,'is', prod_type]
                ]
    ver_ent = sg.find_one( 'Version', ver_find )
    if ver_ent != None:
        ver_ent['exitst'] = 'True'
        return ver_ent
    find_lev3 = [
                ['project.Project.name','is',show],
                ['content','is', lev3],
                ['entity.%s.code'% lev,'is',lev2],
                #['entity.%s.sg_prod_type'% lev,'is',prod_type ],
    ]
#     if prod_type == 'previz':
#         find_lev3.append( ['step.Step.code', 'is' ,'Previz' ] )
#     else:
#         find_lev3.append( ['step.Step.code', 'is_not' ,'Previz' ] )

    lev3_ent = sg.find_one( 'Task', find_lev3 )
    find_lev2 = [
                    ['project.Project.name','is',show],
                    ['code','is',lev2],
                    #['sg_prod_type','is',prod_type],
                ]

    lev2_ent = sg.find_one( lev, find_lev2 )
    username = sg.find_one( 'HumanUser',[['login','is',login]] )
    project_ent = sg.find_one( 'Project',[['name','is',show]] )
    ver_key = {
            'project' : project_ent,
             'code'   : basename,
             'entity' : lev2_ent,
             'sg_task':lev3_ent,
             'sg_status_list': status,
             'description':description,
             'user' : username,
    }

    if path_to_frames:
        ver_key['sg_path_to_frames'] = path_to_frames
    else:
        ver_key['sg_path_to_movie'] = file_path

    ver_ent = sg.create( 'Version', ver_key )
    return ver_ent


print(
                create_version( 'RND',  'shot', 'AAA_0030', 'anim', 'AAA_0030_anim_v04', 
                               '/show/RND/seq/AAA/AAA_0030/anim/preview/AAA_0030_anim_v04.mp4',
                    'onetera', path_to_frames = '', description='', status='wip' )
)

{'sg_path_to_movie': '/show/RND/seq/AAA/AAA_0030/anim/preview/AAA_0030_anim_v04.mp4', 'code': 'AAA_0030_anim_v04', 'description': None, 'entity': {'type': 'Shot', 'id': 126224, 'name': 'AAA_0030'}, 'project': {'type': 'Project', 'id': 686, 'name': 'RND'}, 'sg_status_list': 'wip', 'user': {'type': 'HumanUser', 'id': 993, 'name': '\xec\x98\xa4\xed\x98\xb8\xec\xa4\x80'}, 'type': 'Version', 'id': 82416, 'sg_task': {'type': 'Task', 'id': 28125, 'name': 'anim'}}


In [ ]:
filters = [
            ['attribute_name','is','sg_status_list'],
        ['event_type','is','Shotgun_Version_Change'],
]

In [6]:
import os
print( os)

<module 'os' from '/home/w10137/anaconda3/lib/python3.8/os.py'>


In [14]:
import shotgun_api3 as sa
import os
import imp

from pprint import pprint

sg = sa.Shotgun(
    'https://wswg.shotgunstudio.com',
    api_key='d4f25676239bff829454edd3a5d97bac13c6d38c526367f9623286c4775ca4dd',
    script_name = 'utils',
)
# if plugin:
#     print 'defined plugin'
#     del plugin
# path = os.path.join( os.getcwd(),'plugins','status_event.py' )
# plugin = imp.load_source( 'status_event', path )
# print plugin, dir(plugin)

# pprint(sg.schema_field_read( 'Shot'))
# shots = sg.find_one( 'Shot',[
#         ['project.Project.name','is','RND'],
#         ['code','is','XXX_0030']],['tasks'])
# pprint(shots)

filters = [
    ['attribute_name','is','sg_status_list'],
    ['project.Project.name','not_in',['_Pipeline']],
    ['event_type','in',['Shotgun_Shot_Change','Shotgun_Asset_Change'] ],
    
]
filters = [
    ['id','is',21199229]
]
keys = [
    'created_at','description', 'entity','project.Project.name','code',
    'tasks','meta',
]

# results = sg.find_one(
#     'EventLogEntry', filters , keys
# )
# # print results
# for result in results:
#     print result,' : ', results[result]
# #     if result['meta']['new_value'] == 'omit':
# #         pprint( result)


import datetime as dt
import time

def sync_version_to_task( old_id ):
#    if MAIN_ID == old_id:
#        print "Previous ID is same with old id"
#        return

    filters = [
        ['attribute_name','is','sg_status_list'],
        ['event_type','is','Shotgun_Version_Change'],
    ]
    keys = [
        'created_at','description', 'entity','project.Project.name',
        'entity.Version.sg_task',
        'entity.Version.sg_task.Task.sg_status_list',
        'entity.Version.sg_status_list',
    ]
    if not old_id:
        today = dt.datetime(
            dt.datetime.now().year,
            dt.datetime.now().month,
            dt.datetime.now().day,
#            dt.datetime.now().hour,
#            dt.datetime.now().minute
        )
        filters.append( ['created_at','greater_than',today] )
    else:
        filters.append( ['id', 'greater_than', old_id ] )

    result = sg.find_one(
    #results = sg.find(
        'EventLogEntry', filters , keys
    )
    if not result:
        result = {}
        result['id'] = old_id
        return old_id
    #for result in results:
    if old_id == result['id']:
        print "Previous ID is same with old id"
        result = {}
        result['id'] = old_id
        return old_id
    # pprint( result )
    # print '\n'

    if result and result['entity'] and result['entity.Version.sg_task']:
        updated = sg.update(
                    'Task', result['entity.Version.sg_task']['id'],
                    {'sg_status_list':result['entity.Version.sg_status_list'] }
                   )
        if updated:
            created = result['created_at'].strftime('%Y-%m-%d %H:%M:%S')
            print '[Version status update Task status]','*'*50
            print '{:15} : {}'.format( 'ID'         , result['id'] )
            print '{:15} : {} / {}'.format( 'Project'    , result['project.Project.name'],
                                            result['entity']['name']  )
            print '{:15} : {}'.format( 'created_at' , created )
            print '{:15} : {}'.format( 'Description', result['description'] )
            print 'https://wswg.shotgunstudio.com/detail/task/{}'.format( result['entity.Version.sg_task']['id'] )
            print '\n'

            return result['id']
        else:
            print "[ No Updated ]"
            return result['id']
    else:
        pprint( result )
        print "entity or Version.sg_task is none type"
        return result['id']






print sync_version_to_task( 21199229 )



Fault: API update() CRUD ERROR #5: Update failed for [Task.sg_status_list]: 'csrs' is not a valid status. Valid statuses: 'wtg', 'wip', 'rev', 'ledok', 'csvok', 'svok', 'hold', 'omit', 'retake', 'tpub', 'pub', 'dirok', 'cmpt', 'cntapr', 'tdeli', 'kbk', 'rdy'.

In [3]:
import collections
from pprint import pprint


# namedtuple()
Person = collections.namedtuple("Person", 'name age gender')

P1 = Person(name='Jhon', age=28, gender='남')
pprint( P1 )

Person(name='Jhon', age=28, gender='\xeb\x82\xa8')


In [35]:
import imp

class Plugin:
    def __init__(self):
        self.plugin = imp.load_source( "status_event","./plugins/status_event.py")
    def __name__(self):
        return self.plugin.__name__
pl = Plugin()
print pl.__name__()


status_event
